<a href="https://colab.research.google.com/github/shubham14yadav/A-B-Testing-Case-study/blob/main/A_B_testing_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A/B testing at Vungle - case study

In [ ]:
df = read.csv('case_study_AB_testing_at_Vungle.csv')
head(df)
tail(df)

,Strategy,Date,Impressions,Completes,Clicks,Installs,eRPM
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Vungle A,1-Jun-14,6777407,5978434,345309,31119,3.33
2,Vungle A,2-Jun-14,6004310,5331727,299732,24601,2.94
3,Vungle A,3-Jun-14,5832627,5193549,291384,24220,3.03
4,Vungle A,4-Jun-14,5875702,5227917,295099,23382,2.99
5,Vungle A,5-Jun-14,6843405,6111378,339529,27725,3.08
6,Vungle A,6-Jun-14,7790350,6981471,392987,31820,3.14


,Strategy,Date,Impressions,Completes,Clicks,Installs,eRPM
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
55,Vungle B,25-Jun-14,511505,451388,24894,1725,3.41
56,Vungle B,26-Jun-14,508097,448333,25111,1773,3.72
57,Vungle B,27-Jun-14,518004,457335,25832,1852,3.94
58,Vungle B,28-Jun-14,562854,494686,28491,2041,4.07
59,Vungle B,29-Jun-14,583732,510194,29483,2168,4.05
60,Vungle B,30-Jun-14,537433,470054,26669,1910,3.69


### 1) What are the treatment indicator, Response variables and baseline variables (if any)?

Treatment Indicator - Strategy

Reponse variables - Impressions, Completes, Clicks, Installs, eRPM

Baseline variables - None

### 2)	How were the users assigned to algorithm A and B? What was the proportion of the users assigned to each group?

Using MD5 hashing algorithm

An MD5 hash transforms each user ID into a unique 32-character hexadecimal string. Each character of the hexadecimal string could be 0–9, A, B, C, D, E, or F—16 options in total. Each character occurred with equal likelihood, making it simple for Vungle to direct traffic in 1/16th increments using a logic statement (assuming that the original string was random).

Algorithm A -> 15/16th of users

Algorithm B -> 1/16th of users

### 3) Which metric would be a good choice to measure the effectiveness of the app-promotion campaign and the success of the serving platform?

eRPM (effective revenue) per 1,000 impressions

### 4) What is the additional value provided by the new algorithm compared to the old algorithm in a month?

In [ ]:
# approach 1

# if all the users were exposed with Algorithm B
impressions_B_all_users = 16*sum(df[df$Strategy=='Vungle B',3])

# eRPM from Algorithm B
eRPM_B_all_users = mean(df[df$Strategy=='Vungle B',7])*impressions_B_all_users/1000

# if all the users were exposed with Algorithm A
impressions_A_all_users = sum(df[df$Strategy=='Vungle A',3]) + (1/16)*sum(df[df$Strategy=='Vungle A',3])

# eRPM from Algorithm A
eRPM_A_all_users = mean(df[df$Strategy=='Vungle A',7])*impressions_A_all_users/1000

# additional value provided by the new algorithm
additional_value_of_B = eRPM_B_all_users - eRPM_A_all_users
additional_value_of_B

[1] 34611.31

OR

In [ ]:
# approach 2

avg_daily_eRPM = mean(df[df$Strategy=='Vungle B',7] - df[df$Strategy=='Vungle A',7])

total_impressions = sum(df[df$Strategy=='Vungle B',3] + df[df$Strategy=='Vungle A',3])

additional_value_from_B = total_impressions/1000 * avg_daily_eRPM

additional_value_from_B

[1] 27919.52

### 5)	What assumptions did you make to answer question 4?

* There is no selection bias between the two user groups, i.e., in average, they are identical to eachother.
* (approach 1) If we run algorithm A on the whole users instead of to 15/16th of the users, the business metrics like Impressions, Clicks, Installs, etc. will scale proportionally. This will apply for algorithm B as well if it were run on the whole users.
* (approach 2) Sum of total impressions from algorithm A and B is a fair representation of the total impressions for algorithm B if it were run on the whole users.

### 6)	Assuming all months are like June, can we expect positive benefits from the new algorithm in future months as well?

* **a)	Use 2-sample t-test of difference of means of eRPM to support your answer.**

In [ ]:
t.test(df[df$Strategy=='Vungle B',7], df[df$Strategy=='Vungle A',7])


	Welch Two Sample t-test

data:  df[df$Strategy == "Vungle B", 7] and df[df$Strategy == "Vungle A", 7]
t = 1.492, df = 48.739, p-value = 0.1421
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.03840841  0.25974174
sample estimates:
mean of x mean of y 
 3.459333  3.348667 


We cannot reject the null hypothesis that there is no difference in the means. Thus, based on the results from 2-sample t-test of difference of means of eRPM, we **cannot** expect positive benefits form the new algorithm in future months.

* **b)	Use paired t-test for the difference of means of eRPM to support your answer.**

In [ ]:
t.test(df[df$Strategy=='Vungle B',7], df[df$Strategy=='Vungle A',7], paired = TRUE)


	Paired t-test

data:  df[df$Strategy == "Vungle B", 7] and df[df$Strategy == "Vungle A", 7]
t = 3.2837, df = 29, p-value = 0.002677
alternative hypothesis: true mean difference is not equal to 0
95 percent confidence interval:
 0.04173767 0.17959566
sample estimates:
mean difference 
      0.1106667 


The p-value for the paired t-test is less than 0.05, so we have enough evidence to reject the null hypothesis. Thus, there is a statistical difference between the eRPM from Algorithm B and Algorithm A. And since the mean difference of eRPM between algorithm B and algorithm A is positive, we can say that Algorithm B is better than Algorithm A.

* **c)	Do you get the same conclusion from a. and b.?**

No, the conclusions from 2-sample t-test and paired t-test for the difference of means of eRPM are not the same. 2-sample t-test concludes that there is no statistical difference between Algorithm A and B, while paired t-test concludes that Algorithm B is better than Algorithm A.

* **d)	Which test (2-sample t-test vs paired t-test) would be more appropriate in the given situation and why?**

Paired t-test would be more appropriate for the situation.

Even though we have two groups of users (users exposed with Algorithm A and users exposed with Algorithm B), they are, for our purpose, identical in average because the users were randomly assigned to group A or B. In average all the features between these two group, expect for the exposure of the algorithm, are the same. Thus, the two observations of metrics on each day can be considered as paired.

### 7)	Compute the conversion rates for algorithm A and B (refer to Figure 1. in the case study for the definition of conversion rate)?

In [ ]:
total_installs_A = sum(df[df$Strategy=='Vungle A',6])
total_installs_B = sum(df[df$Strategy=='Vungle B',6])
total_impressions_A = sum(df[df$Strategy=='Vungle A',3])
total_impressions_B = sum(df[df$Strategy=='Vungle B',3])

conversion_rate_A = total_installs_A / total_impressions_A
cat('Conversion rate of A:', conversion_rate_A , '\n')
conversion_rate_B = total_installs_B / total_impressions_B
cat('Conversion rate of B:', conversion_rate_B)

Conversion rate of A: 0.004024649 
Conversion rate of B: 0.003540579

### 8) Which algorithm is better based on the conversion rates? Use proportions test to support your answer.

In [ ]:
prop.test(c(total_installs_A, total_installs_B), c(total_impressions_A, total_impressions_B), correct = FALSE)


	2-sample test for equality of proportions without continuity correction

data:  c(total_installs_A, total_installs_B) out of c(total_impressions_A, total_impressions_B)
X-squared = 873.64, df = 1, p-value < 2.2e-16
alternative hypothesis: two.sided
95 percent confidence interval:
 0.0004537127 0.0005144257
sample estimates:
     prop 1      prop 2 
0.004024649 0.003540579 


The p-value of the proportions test on the conversion rates is less than 0.05. Thus, we can reject the null hypothesis that there is no difference between the proportions of the conversion rates from Algorithm A and Algorithm B. Since coversion rate of Algorithm A be larger than that of Algorithm B, we can conclude that Algorithm A better than Algorithm B (based on Coversion rates).

### 9)	The additional monthly operating cost of Algorithm B compared to Algorithm A is $2000. Use the confidence interval from the paired t-test in question 6.b. to decide whether Algorithm B should replace Algorithm A.

From the paired t-test above, the Confidence interval is [0.04173767 , 0.17959566]

We can use the lower bound of the confidence interval to calculate the minimum lift we can expect from Algorithm B.

In [ ]:
# approach 1
lower_bound_additional_income_B = 0.04173767 * impressions_B_all_users / 1000
lower_bound_additional_income_B

[1] 10568.23

OR

In [ ]:
# approach 2
lower_bound_additional_income_B = 0.04173767 * total_impressions / 1000
lower_bound_additional_income_B

[1] 10529.78

The minimum lift we get from Algorithm B is more than 10500 dollars, while it only costs additional 2000 dollars to maintain. Thus, we should definitely replace Algorithm A by Algorithm B (>8500 dollars additional revenue per month).